In [19]:
%load_ext autoreload
%autoreload 2

In [20]:
%cd /home/aditya/git/kaggle_pneumonia 
%env PROJECT_PATH = /home/aditya/git/kaggle_pneumonia 

/home/aditya/git/kaggle_pneumonia
env: PROJECT_PATH=/home/aditya/git/kaggle_pneumonia


In [21]:
%matplotlib inline
import seaborn as sns

In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

In [41]:
from torchsummary import summary

In [57]:
import pandas as pd
import numpy as np
import pydicom
from PIL import Image
import multiprocessing as mp
from tqdm import tqdm_notebook as tqdm

import torch.optim as optim
import torchvision
from sklearn.model_selection import train_test_split

from utils.envs import *
from utils.data_load import *
from utils.lr_finder import lr_plot, lr_find
from utils.common import get_batch_info
from utils.checkpoint import save_checkpoint, load_cp_model, load_cp_optim
from utils.logger import logger

from model.arch.pneunetv1 import PneuNetv1, PneuNetHeader
from model.dataset import Single_Pneumonia_Dataset, Single_Pneumonia_Dataset_Test
from model.criterion import BoundBoxCriterion
from model.train import train_step, fit_model
from model.validation import validate_model
from model.test import predict_model
from model.optim import CLR

from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [58]:
from torch import nn as nn
from utils.common import to_list
from model.arch.pneunetv1 import PneuNetHeader
from model.arch.pneunetv2 import PneuNetv2

In [59]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [60]:
preload_model = torchvision.models.resnet34(pretrained=True).to(device)
header_model = PneuNetHeader(512, 5, [1000, 1000, 128], 0.5).to(device)

In [61]:
model = PneuNetv2(preload_model, header_model).to(device)

In [62]:
train_label_df = pd.read_csv(train_label_repo)
train_label_df['area'] = train_label_df.width * train_label_df.height
single_label_df = train_label_df.sort_values('area', ascending = False).drop_duplicates('patientId').sort_index().reset_index(drop = True).copy()
single_label_df.fillna(0, inplace = True)

In [63]:
train_idx = np.arange(len(single_label_df))
dev_idx, val_idx = train_test_split(train_idx, test_size = 0.20)

In [64]:
dev_df = single_label_df.iloc[dev_idx,:].reset_index(drop = True)
val_df = single_label_df.iloc[val_idx,:].reset_index(drop = True)

In [65]:
dev_dataset = Single_Pneumonia_Dataset(dev_df, train_dcm_path, device)
val_dataset = Single_Pneumonia_Dataset(val_df, train_dcm_path, device)
test_dataset = Single_Pneumonia_Dataset_Test(test_dcm_path, device)

In [66]:
dev_dataloader = DataLoader(dev_dataset, batch_size = 32, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 32)
test_dataloader = DataLoader(test_dataset, batch_size = 32)

In [68]:
n_epoch = 51
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion = BoundBoxCriterion(alpha = 0.5).to(device)

In [69]:
n_obs, batch_size, n_batch_per_epoch = get_batch_info(dev_dataloader)
clr = CLR(optimizer, n_epoch, n_batch_per_epoch, 0.1, 1., 0.95, 0.85, 2)
callbacks = [clr]

In [ ]:
model, callbacks = fit_model(model = model, n_epoch = n_epoch, dev_dataloader = dev_dataloader, optimizer = optimizer, criterion = criterion, callbacks = callbacks, val_dataloader = val_dataloader)